In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
data_name = "wipo_d"

In [3]:
with open('data/%s/hierarchy.txt' % data_name, 'w') as f1:
    with open('data/%s/%s.ht' % (data_name,data_name)) as f:
        for l in f:
            split = l.strip().split(':')
            p = split[0]
            c = split[1].split(',')
            for cc in c:
                f1.write('%s %s\n'% (p,cc))

In [4]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

In [5]:
for mode in ['train', 'test']:
    for i in range(1,6):
        datas, labels = preparation.import_data('%s/folds/%s_fold%d.dat.%s' % (data_name, data_name, i, mode))
        new_labels = preparation.map_index_of_label('%s/hierarchy.pickle' % data_name, labels)
        if mode == 'train':
            train_data, validate_data, train_target, validate_target = train_test_split(
                datas, new_labels, test_size=0.1, random_state=12345)
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), train_data, train_target)
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, "validate"), validate_data, validate_target)
        else:
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), datas, new_labels)

In [6]:
for i in range(len(level)-1):
    print(level[i+1] - level[i])

7
20
160
645


# Real Flow 

In [7]:
from data import Dataset
from embedding import Doc2Vec
from assemble_classifier import AssembleNoLabel, AssemblePredicted,AssemblePredictedHidden ,AssemblePredictedHiddenAdd
import shutil
import os

%load_ext autoreload
%autoreload 2

In [8]:
%%time
for fold in range(1, 6):
    print()
    print()
    if os.path.isdir('data/%s/output' % data_name):
        shutil.rmtree('data/%s/output' % data_name)

    dataset_train = Dataset(data_name, fold, "train")
    dataset_validate = Dataset(data_name, fold, "validate")
    dataset_test = Dataset(data_name, fold, "test")

    print("train doc2vec")http://localhost:8888/notebooks/Demo-5fold.ipynb#
    doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=100, epoch=1000)
    doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels)

    dataset_train.change_to_Doc2Vec(doc2vec)
    dataset_validate.change_to_Doc2Vec(doc2vec)
    dataset_test.change_to_Doc2Vec(doc2vec)
    
    print("train main classifier")
    model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=20000, hidden_size=[300,1500,1500,1500], target_hidden_size=[60,60,60], use_dropout=True, start_level=0)
#     model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,2000,2500], use_dropout=False, start_level=0, end_level=100)
    #     model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=20000, hidden_size=[300,1500,2000,2500,2500,1500,1500,1500,300], target_hidden_size=[60,100,150,200,200,150,150,100,60], use_dropout=True, start_level=0)
    model.train()
    model.tuning_threshold()

    print("evaluate")
    f1_macro, f1_micro, f1_each = model.evaluate("train", correction=True)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for level, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))
    f1_macro, f1_micro, f1_each = model.evaluate("validate", correction=True)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for level, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))
    f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for level, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))



train doc2vec
Doc2Vec by Gensim
Epoch: 20 Similar: 2.49
Epoch: 40 Similar: 13.70
Epoch: 60 Similar: 17.03
Epoch: 80 Similar: 17.39
Epoch: 100 Similar: 16.90
Epoch: 120 Similar: 17.30
Epoch: 140 Similar: 16.74
Epoch: 160 Similar: 17.36
Epoch: 180 Similar: 17.95
Epoch: 200 Similar: 17.53
Epoch: 220 Similar: 17.23
Epoch: 240 Similar: 17.70
Stopping Similar: 17.70
train main classifier
Level: 1.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.185 Training F1 macro: 0.984 Validate F1 macro: 0.674
Level: 1.000 Epoch: 1166/20000 Batch: 1/1 Loss: 0.101 Stopping F1 macro: 0.994 Validate F1 macro: 0.706

Level: 2.000 Epoch: 666/20000 Batch: 1/1 Loss: 0.124 Training F1 macro: 0.997 Validate F1 macro: 0.517
Level: 2.000 Epoch: 1166/20000 Batch: 1/1 Loss: 0.055 Stopping F1 macro: 1.000 Validate F1 macro: 0.554

Level: 3.000 Epoch: 666/20000 Batch: 1/1 Loss: 1.052 Training F1 macro: 0.841 Validate F1 macro: 0.052
Level: 3.000 Epoch: 1332/20000 Batch: 1/1 Loss: 0.377 Training F1 macro: 0.940 Validate F1 mac

Level: 3.000 Epoch: 2664/20000 Batch: 1/1 Loss: 0.510 Training F1 macro: 0.918 Validate F1 macro: 0.072
Level: 3.000 Epoch: 3255/20000 Batch: 1/1 Loss: 0.432 Stopping F1 macro: 0.928 Validate F1 macro: 0.081

Level: 4.000 Epoch: 666/20000 Batch: 1/1 Loss: 3.148  Training F1 macro: 0.063 Validate F1 macro: 0.001
Level: 4.000 Epoch: 1332/20000 Batch: 1/1 Loss: 1.720 Training F1 macro: 0.429 Validate F1 macro: 0.004
Level: 4.000 Epoch: 1998/20000 Batch: 1/1 Loss: 1.161 Training F1 macro: 0.693 Validate F1 macro: 0.009
Level: 4.000 Epoch: 2664/20000 Batch: 1/1 Loss: 0.806 Training F1 macro: 0.782 Validate F1 macro: 0.009
Level: 4.000 Epoch: 2665/20000 Batch: 1/1 Loss: 0.831 Stopping F1 macro: 0.781 Validate F1 macro: 0.011

evaluate
F1 macro: 0.6562 F1 micro: 0.6824
Level: 0 F1 macro: 0.8903 F1 micro: 0.9245
Level: 1 F1 macro: 0.7689 F1 micro: 0.8007
Level: 2 F1 macro: 0.6959 F1 micro: 0.5833
Level: 3 F1 macro: 0.6404 F1 micro: 0.5499
F1 macro: 0.0511 F1 micro: 0.3595
Level: 0 F1 macro: 0.

In [9]:
import tensorflow as tf

/home/chanatip/miniconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/chanatip/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
